In [2]:
import torch
import numpy as np
from transformers import T5EncoderModel, T5Tokenizer
from sklearn.cluster import KMeans, DBSCAN
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import hdbscan
from collections import defaultdict

device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')
print("Using device: {}".format(device))


def get_T5_model():
    local_model_path = "/home/lww/prot_t5_model"  # 指向您下载模型的路径
    print("Loading from local path: {}".format(local_model_path))
    
    model = T5EncoderModel.from_pretrained(local_model_path, local_files_only=True)
    tokenizer = T5Tokenizer.from_pretrained(local_model_path, do_lower_case=False, local_files_only=True)
    model = model.to(device)
    model = model.eval()
    return model, tokenizer


def get_embedding(sequence, model, tokenizer):
    """获取单个蛋白质序列的嵌入向量"""
    # 处理特殊氨基酸
    sequence = sequence.replace('U', 'X').replace('Z', 'X').replace('O', 'X')

    # 检测是使用ProtT5还是ProtBERT
    is_t5 = isinstance(model, T5EncoderModel)

    if is_t5:
        # 氨基酸之间加空格 (T5需要)
        sequence = ' '.join(list(sequence))

    # 编码序列
    inputs = tokenizer(sequence, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 获取嵌入向量
    with torch.no_grad():
        if is_t5:
            embedding_repr = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
            # T5返回的是last_hidden_state
            emb = embedding_repr.last_hidden_state.mean(dim=1)  # 对所有token进行平均
        else:
            # ProtBERT
            embedding_repr = model(**inputs)
            # 使用[CLS]标记的表示
            emb = embedding_repr.last_hidden_state[:, 0, :]

    return emb.cpu().numpy().squeeze()


def process_protein_set(protein_list, batch_size=32):
    """批量处理蛋白质序列并生成嵌入向量"""
    model, tokenizer = get_T5_model()
    embeddings = []

    # 批量处理以提高效率
    for i in range(0, len(protein_list), batch_size):
        batch = protein_list[i:i + batch_size]
        print(f"Processing batch {i // batch_size + 1}/{len(protein_list) // batch_size + 1}")

        for protein in batch:
            try:
                embedding = get_embedding(protein, model, tokenizer)
                embeddings.append(embedding)
            except Exception as e:
                print(f"Error processing protein: {e}")
                # 添加一个空向量作为占位符
                # 使用768或1024维度取决于使用的是ProtBERT还是ProtT5
                dim = 1024 if isinstance(model, T5EncoderModel) else 768
                embeddings.append(np.zeros(dim))

    return np.array(embeddings)
    
def main(protein_list):
    """处理蛋白质列表并进行聚类分析"""
    # 1. 生成嵌入向量
    print("生成蛋白质嵌入向量...")
    embeddings = process_protein_set(protein_list)
    return embeddings


# 主程序示例，修正集合不可索引的问题
def run_analysis(df):
    # 提取蛋白质序列集合
    drug_set = set()
    protein_set = set()
    for i in range(len(df)):
        drug_set.add(df.loc[i, 'compound_iso_smiles'])
        protein_set.add(df.loc[i, 'target_sequence'])

    print(f"药物数量: {len(drug_set)}")
    print(f"蛋白质数量: {len(protein_set)}")

    # 将集合转换为列表
    protein_list = list(protein_set)

    # 调用主函数进行分析
    embeddings = main(protein_list)
    return embeddings, protein_list

Using device: cuda:7


In [3]:
import numpy as np
import pickle
import os

def save_embeddings_and_proteins(embeddings, protein_list, output_dir="saved_data"):
    """
    保存嵌入向量和蛋白质序列到文件
    
    参数:
        embeddings: 蛋白质的嵌入向量数组
        protein_list: 对应的蛋白质序列列表
        output_dir: 保存目录
    """
    # 创建目录
    os.makedirs(output_dir, exist_ok=True)
    
    # 保存embeddings (numpy array)
    np.save(os.path.join(output_dir, "protein_embeddings.npy"), embeddings)
    
    # 保存protein_list (使用pickle)
    with open(os.path.join(output_dir, "protein_list.pkl"), "wb") as f:
        pickle.dump(protein_list, f)
    
    print(f"数据已保存到 {output_dir} 目录")
    print(f"- 嵌入向量: {embeddings.shape}")
    print(f"- 蛋白质序列: {len(protein_list)}")

def load_embeddings_and_proteins(input_dir="saved_data"):
    """
    从文件加载嵌入向量和蛋白质序列
    
    参数:
        input_dir: 保存目录
        
    返回:
        embeddings: 蛋白质的嵌入向量数组
        protein_list: 对应的蛋白质序列列表
    """
    # 加载embeddings
    embeddings_path = os.path.join(input_dir, "protein_embeddings.npy")
    embeddings = np.load(embeddings_path)
    
    # 加载protein_list
    protein_list_path = os.path.join(input_dir, "protein_list.pkl")
    with open(protein_list_path, "rb") as f:
        protein_list = pickle.load(f)
    
    print(f"数据已从 {input_dir} 加载")
    print(f"- 嵌入向量: {embeddings.shape}")
    print(f"- 蛋白质序列: {len(protein_list)}")
    
    return embeddings, protein_list

In [4]:
import pandas as pd
import os
df = pd.read_csv(os.path.join('/home/lww/learn_project/MGraphDTA-dev/regression/data/kiba/raw/data.csv'))
embeddings, protein_list = run_analysis(df)
save_embeddings_and_proteins(embeddings, protein_list, output_dir="saved_protein_data")

药物数量: 1767
蛋白质数量: 1876
生成蛋白质嵌入向量...
Loading from local path: /home/lww/prot_t5_model


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Processing batch 1/59
Processing batch 2/59
Processing batch 3/59
Processing batch 4/59
Processing batch 5/59
Processing batch 6/59
Processing batch 7/59
Processing batch 8/59
Processing batch 9/59
Processing batch 10/59
Processing batch 11/59
Processing batch 12/59
Processing batch 13/59
Processing batch 14/59
Processing batch 15/59
Processing batch 16/59
Processing batch 17/59
Processing batch 18/59
Processing batch 19/59
Processing batch 20/59
Processing batch 21/59
Processing batch 22/59
Processing batch 23/59
Processing batch 24/59
Processing batch 25/59
Processing batch 26/59
Processing batch 27/59
Processing batch 28/59
Processing batch 29/59
Processing batch 30/59
Processing batch 31/59
Processing batch 32/59
Processing batch 33/59
Processing batch 34/59
Processing batch 35/59
Processing batch 36/59
Error processing protein: CUDA out of memory. Tried to allocate 20.46 GiB. GPU 7 has a total capacity of 23.64 GiB of which 12.66 GiB is free. Including non-PyTorch memory, this pro

In [4]:
# 不再需要运行模型，直接从文件加载
embeddings, protein_list = load_embeddings_and_proteins(input_dir="saved_protein_data")

数据已从 saved_protein_data 加载
- 嵌入向量: (1876, 1024)
- 蛋白质序列: 1876


In [5]:
import numpy as np
import pandas as pd
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from collections import defaultdict
import os

def run_balanced_spectral_clustering(embeddings, protein_list, output_dir="balanced_spectral_results"):
    """
    实现先前更均衡的谱聚类结果
    
    参数:
        embeddings: 蛋白质的嵌入向量数组
        protein_list: 对应的蛋白质序列列表
        output_dir: 结果输出目录
    
    返回:
        labels: 聚类标签
        clusters: 聚类结果字典
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # 步骤1: 使用UMAP进行降维，这是获得更均衡结果的关键
    print("执行UMAP降维...")
    import umap
    reducer = umap.UMAP(
        n_neighbors=30,       # 较大的邻居数可以捕获更全局的结构
        min_dist=0.1,         # 中等的min_dist值
        n_components=20,      # 降至较高维度
        metric='cosine',      # 使用余弦相似度
        random_state=42
    )
    umap_data = reducer.fit_transform(embeddings)
    print(f"UMAP降维: 从{embeddings.shape[1]}维减至{umap_data.shape[1]}维")
    
    # 步骤2: 应用谱聚类，关键参数不同
    print("执行谱聚类...")
    clustering = SpectralClustering(
        n_clusters=25,
        affinity='nearest_neighbors',
        n_neighbors=15,       # 较大的邻居数
        assign_labels='kmeans',  # 使用kmeans分配标签
        random_state=42,
        n_jobs=-1
    )
    
    labels = clustering.fit_predict(umap_data)
    
    # 计算轮廓系数
    score = silhouette_score(umap_data, labels)
    print(f"聚类完成，轮廓系数: {score:.4f}")
    
    # 将蛋白质分组到各个聚类中
    clusters = defaultdict(list)
    
    for i, label in enumerate(labels):
        clusters[label].append(protein_list[i])
    
    # 计算每个聚类的大小
    cluster_sizes = {label: len(proteins) for label, proteins in clusters.items()}
    sorted_clusters = sorted(cluster_sizes.items(), key=lambda x: x[1], reverse=True)
    
    print("\n===== 聚类分析结果 =====")
    print(f"总蛋白质数量: {len(protein_list)}")
    print(f"总聚类数量: {len(clusters)}")
    
    # 显示每个聚类的大小
    for label, size in sorted_clusters:
        print(f"聚类 {label}: {size} 个样本 ({size/len(protein_list)*100:.1f}%)")
    
    # 导出每个聚类的蛋白质序列
    fasta_dir = os.path.join(output_dir, "fasta_files")
    os.makedirs(fasta_dir, exist_ok=True)
    
    for label, proteins in clusters.items():
        # 限制大聚类的序列数量
        export_proteins = proteins[:min(100, len(proteins))]
        
        # 写入FASTA文件
        with open(os.path.join(fasta_dir, f"cluster_{label}.fasta"), "w") as f:
            for i, seq in enumerate(export_proteins):
                f.write(f">protein_{label}_{i+1}\n{seq}\n")
    
    return labels, clusters

In [6]:
# 假设embeddings和protein_list已加载
labels, clusters = run_balanced_spectral_clustering(
    embeddings=embeddings,
    protein_list=protein_list,
    output_dir="balanced_spectral_results"
)

# 获取特定聚类的蛋白质
cluster_id = 2  # 假设我们想查看第2号聚类
proteins = clusters[cluster_id]

print(f"聚类 {cluster_id} 包含 {len(proteins)} 个蛋白质")

执行UMAP降维...


/home/lww/anaconda3/envs/bio/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/lww/anaconda3/envs/bio/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP降维: 从1024维减至20维
执行谱聚类...


/home/lww/anaconda3/envs/bio/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


聚类完成，轮廓系数: 0.3281

===== 聚类分析结果 =====
总蛋白质数量: 1876
总聚类数量: 25
聚类 8: 228 个样本 (12.2%)
聚类 3: 209 个样本 (11.1%)
聚类 18: 200 个样本 (10.7%)
聚类 1: 144 个样本 (7.7%)
聚类 2: 106 个样本 (5.7%)
聚类 20: 101 个样本 (5.4%)
聚类 0: 95 个样本 (5.1%)
聚类 23: 84 个样本 (4.5%)
聚类 6: 77 个样本 (4.1%)
聚类 22: 70 个样本 (3.7%)
聚类 5: 65 个样本 (3.5%)
聚类 4: 53 个样本 (2.8%)
聚类 16: 51 个样本 (2.7%)
聚类 17: 47 个样本 (2.5%)
聚类 12: 47 个样本 (2.5%)
聚类 11: 46 个样本 (2.5%)
聚类 13: 43 个样本 (2.3%)
聚类 14: 38 个样本 (2.0%)
聚类 24: 33 个样本 (1.8%)
聚类 15: 31 个样本 (1.7%)
聚类 19: 30 个样本 (1.6%)
聚类 21: 29 个样本 (1.5%)
聚类 7: 18 个样本 (1.0%)
聚类 9: 16 个样本 (0.9%)
聚类 10: 15 个样本 (0.8%)
聚类 2 包含 106 个蛋白质


In [17]:
cluster_sizes = {label: len(proteins) for label, proteins in clusters.items()}
cluster_sizes

{4: 53,
 16: 51,
 1: 144,
 3: 209,
 10: 15,
 14: 38,
 20: 101,
 8: 228,
 22: 70,
 6: 77,
 18: 200,
 0: 95,
 19: 30,
 24: 33,
 2: 106,
 13: 43,
 17: 47,
 12: 47,
 7: 18,
 5: 65,
 11: 46,
 23: 84,
 21: 29,
 9: 16,
 15: 31}

In [7]:
import os
import random
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.cluster import KMeans
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold
from scipy.cluster.hierarchy import dendrogram, linkage, leaves_list, cut_tree
from rdkit.Chem import AllChem
from collections import Counter
import pandas as pd
import numpy as np
from collections import defaultdict
import joblib
import json

class ScaffoldGenerator(object):
    def __init__(self, include_chirality=False):
        self.include_chirality = include_chirality

    def get_scaffold(self, mol):
        return MurckoScaffold.MurckoScaffoldSmiles(
            mol=mol, includeChirality=self.include_chirality)


def generate_scaffold(smiles, include_chirality=False):
    mol = Chem.MolFromSmiles(smiles)
    engine = ScaffoldGenerator(include_chirality=include_chirality)
    scaffold = engine.get_scaffold(mol)
    return scaffold

def search_index(unique_smiles, df, num_class, num_limit):
    vec_list = []
    for smi in unique_smiles:
        m1 = Chem.MolFromSmiles(smi)
        fp4 = list(AllChem.GetMorganFingerprintAsBitVect(m1, radius=2, nBits=256))
        vec_list.append(fp4)
    print("drug num", len(vec_list))
    Z = linkage(vec_list, 'average', metric='jaccard')
    cluster = cut_tree(Z, num_class).ravel()
    stat_dict = {k: v for k, v in sorted(Counter(cluster).items(), key=lambda item: item[1], reverse=True)}

    num = 0
    data_dict = defaultdict(list)
    for k,v in stat_dict.items():
        pos = np.nonzero(cluster==k)[0]
        # print(k, stat_dict[k], len(pos))
        smi_idx = []
        for idx in pos:
            smi_single = df[df["compound_iso_smiles"] == unique_smiles[idx]]
            smi_idx.append(smi_single)
        df_tmp = pd.concat(smi_idx)
        num += len(df_tmp)
        data_dict[k] = df_tmp
    print("@@@@@@@@@@@", len(data_dict.keys()), num)

    num = 0
    all_keys = list(data_dict.keys())
    class_num = -1
    meat_class = {}
    for k,v in data_dict.items():
        if len(v) > num_limit:
            class_num += 1
            meat_class[class_num] = v
            num += len(v)
            all_keys.remove(k)
    random.shuffle(all_keys)

    smi_idx = []
    smi_idx_num = 0
    for i,k in enumerate(all_keys):
        # print(i, len(data_dict[k]))
        if smi_idx_num < num_limit:
            smi_idx.append(data_dict[k])
            smi_idx_num += len(data_dict[k])
        else:
            class_num += 1
            meat_class[class_num] = pd.concat(smi_idx)
            num += len(meat_class[class_num])

            smi_idx = []
            smi_idx_num = 0
            smi_idx.append(data_dict[k])
            smi_idx_num += len(data_dict[k])
        if i == len(all_keys) -1:
            class_num += 1
            meat_class[class_num] = pd.concat(smi_idx)
            num += len(meat_class[class_num])

    print(class_num, len(meat_class[class_num]),num)

    if len(meat_class[class_num]) < 10:
        meat_class.pop(class_num)

    num = 0
    for k,v in meat_class.items():
        num += len(v)
    print(num)
    return meat_class

In [8]:
df = pd.read_csv(os.path.join('/home/lww/learn_project/MGraphDTA-dev/classification/data/celegans/raw/data.csv'))
scaffolds = {}
protberts = {}
print(len(df))


drug_set = set()
protein_set = set()
for i in range(len(df)):
    drug_set.add(df.loc[i, 'compound_iso_smiles'])
    protein_set.add(df.loc[i, 'target_sequence'])  # smile

print(len(drug_set))
print(len(protein_set))

for d in drug_set:
    try:
        scaffold = generate_scaffold(d)
        if scaffolds.__contains__(scaffold):
            scaffolds[scaffold] = scaffolds[scaffold] + 1
        else:
            scaffolds[scaffold] = 1
    except:
        print("error", d)
        # df.drop(index=i, inplace=True)
        continue

cluster_sizes = {label: len(proteins) for label, proteins in clusters.items()}

7786
1767
1876


[10:35:37] WARNING: not removing hydrogen atom without neighbors


In [9]:
smile_scafold = {}
for d in drug_set:
    smile_scafold[d] = generate_scaffold(d)

protein_cluster = {}
for i, protein in enumerate(protein_list):
    protein_cluster[protein] = labels[i]

[10:35:42] WARNING: not removing hydrogen atom without neighbors


In [72]:
all_key = scaffolds.keys()
print(len(all_key))  # all_key_drugs
train_scaffold = random.sample(all_key, round(len(all_key) * 0.80))
total_count1 = sum(scaffolds[s] for s in train_scaffold)
print(total_count1/len(drug_set))


# 蛋白质聚类划分等效代码
all_clusters = list(clusters.keys())  # 获取所有聚类标签
print(len(all_clusters))  # 打印聚类数量

# 随机选择88%的聚类作为训练集
train_clusters = random.sample(all_clusters, round(len(all_clusters) * 0.80))

# 计算训练集中的蛋白质数量
total_count_proteins = sum(len(clusters[c]) for c in train_clusters)

# 打印训练集蛋白质占总蛋白质的比例
print(total_count_proteins/len(protein_list))

687
0.8653084323712507
25
0.8768656716417911


/tmp/ipykernel_1976062/2510592015.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  train_scaffold = random.sample(all_key, round(len(all_key) * 0.80))


In [93]:
while True:
    # 1. 药物支架划分
    all_key = list(scaffolds.keys())
    print(len(all_key))  # all_key_drugs
    train_scaffold = random.sample(all_key, round(len(all_key) * 0.80))
    total_count1 = sum(scaffolds[s] for s in train_scaffold)
    print(total_count1 / len(drug_set))

    # 2. 蛋白质聚类划分
    all_clusters = list(clusters.keys())  # 获取所有聚类标签
    print(len(all_clusters))  # 打印聚类数量

    train_clusters = random.sample(all_clusters, round(len(all_clusters) * 0.80))
    total_count_proteins = sum(len(clusters[c]) for c in train_clusters)
    print(total_count_proteins / len(protein_list))

    # 3. 根据支架和聚类进行数据划分
    train_idx = []
    test1_idx = []
    test2_idx = []

    for i in range(len(df)):
        smi = smile_scafold[df.loc[i, 'compound_iso_smiles']]
        prot = protein_cluster[df.loc[i, 'target_sequence']]

        if smi in train_scaffold and prot in train_clusters:
            train_idx.append(i)  # 都在，加入训练集
        elif smi not in train_scaffold and prot not in train_clusters:
            test2_idx.append(i)  # 都不在，加入 test2
        else:
            test1_idx.append(i)  # 一个在一个不在，加入 test1

    print(len(train_idx), len(test1_idx), len(test2_idx), len(train_idx) + len(test1_idx) + len(test2_idx))

    # 4. 如果训练集满足长度要求，则保存数据并退出循环
    if len(train_idx) > (len(train_idx) + len(test1_idx) + len(test2_idx))*0.8):
        df_old = df.loc[train_idx].reset_index(drop=True)
        df_old.to_csv('/home/lww/learn_project/MGraphDTA-dev/classification/data/celegans/train_val.csv')

        df_test1 = df.loc[test1_idx].reset_index(drop=True)
        df_test1.to_csv('/home/lww/learn_project/MGraphDTA-dev/classification/data/celegans/test1.csv')

        df_test2 = df.loc[test2_idx].reset_index(drop=True)
        df_test2.to_csv('/home/lww/learn_project/MGraphDTA-dev/classification/data/celegans/test2.csv')

        print("done")
        break  # 满足条件，退出循环
    else:
        print("训练集样本不足，重新执行划分操作...")

687
0.8834182229767968
25
0.8406183368869936
5752 1901 133 7786
done


In [ ]:
df_train = pd.read_csv('/home/lww/learn_project/MGraphDTA-dev/classification/data/celegans/train_val.csv')
unique_smi = df_train["compound_iso_smiles"].unique()
num_classes = len(unique_smi) // 10
num_limit = len(df_train) // 50
meat_class = search_index(unique_smi, df_train, num_classes, num_limit)
print(len(meat_class.keys()))
meta_train = {}
meta_train_num = 0
meta_train_k_num = 0
meta_val = {}
meta_val_num = 0
meta_val_k_num = 0
meta_keys = list(meat_class.keys())
random.shuffle(meta_keys)
for k in meta_keys:
    if len(meta_train.keys()) < len(meta_keys) *0.8:
        meta_train[k] = meat_class[k]
        meta_train_num += len(meat_class[k])
        meta_train_k_num += 1
    else:
        meta_val_k_num +=1
        meta_val[k] = meat_class[k]
        meta_val_num += len(meat_class[k])

joblib.dump(meta_train, "/home/lww/learn_project/MGraphDTA-dev/classification/data/celegans/meta_train.pkl")
joblib.dump(meta_val, "/home/lww/learn_project/MGraphDTA-dev/classification/data/celegans/meta_val.pkl")

In [77]:
meta_train = joblib.load("/home/lww/learn_project/MGraphDTA-dev/classification/data/celegans/meta_train.pkl")
meta_val = joblib.load("/home/lww/learn_project/MGraphDTA-dev/classification/data/celegans/meta_val.pkl")
train_pd = []
for k,v in meta_train.items():
    train_pd.append(v)
df_tmp = pd.concat(train_pd)
df_tmp = df_tmp.reset_index(drop=True)
df_tmp.to_csv('/home/lww/learn_project/MGraphDTA-dev/classification/data/celegans/train.csv')
print(len(df_tmp))

test_pd = []
for k, v in meta_val.items():
    test_pd.append(v)
df_tmp = pd.concat(test_pd)
df_tmp = df_tmp.reset_index(drop=True)
df_tmp.to_csv('/home/lww/learn_project/MGraphDTA-dev/classification/data/celegans/val.csv')

print(len(df_tmp))

5254
1193
